In [ ]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn.datasets import make_classification
import numpy as np
from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
import pickle
import os
import gzip

np.random.seed(1)
import random

n = 50
X, Y = make_classification(n_samples = n, n_features = 2, n_redundant=0, n_informative=2, flip_y=0)
Y = Y*2-1
X = X.astype(np.float32); Y = Y.astype(np.int32)

train_x, test_x = np.split(X, [n*8//10])
train_labels, test_labels = np.split(Y, [n*8//10])
print("Features:\n",train_x[0:4])
print("Labels:\n", train_labels[0:4])

def plot_dataset(suptitle, features, labels):
    fig, ax = pylab.subplots(1,1)
    fig.suptitle(suptitle, fontsize=16)
    ax.set_xlabel('$x_i[0]$ -- (feature 1)')
    ax.set_ylabel('$x_i[1]$ -- (feature 2)')
    
    colors = ['r' if l>0 else 'b' for l in labels]
    ax.scatter(features[:, 0], features[:, 1], marker='o', c=colors, s=100, alpha=0.5)
    fig.show()

plot_dataset('Training_data', train_x, train_labels)

pos_examples = np.array([ [t[0], t[1], 1] for i,t in enumerate(train_x)
                        if train_labels[i]>0])
neg_examples = np.array([ [t[0], t[1], 1] for i,t in enumerate(train_x)
                         if train_labels[i]<0])

print(pos_examples[0:3])
print(neg_examples[0:3])

def train(positive_examples, negative_examples, num_iterations = 100):
    num_dims = positive_examples.shape[1]
    
    weights = np.zeros((num_dims, 1))
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]
    
    report_frequency = 10
    
    for i in range(num_iterations):
        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)
        
        z = np.dot(pos, weights)
        if z < 0:
            weights = weights + pos.reshape(weights.shape)
            
        z = np.dot(neg, weights)
        if z >= 0:
            weights = weights - neg.reshape(weights.shape)
        
        if i % report_frequency == 0:
            pos_out = np.dot(positive_examples, weights)
            neg_out = np.dot(negative_examples, weights)
            pos_correct = (pos_out >= 0).sum() / float(pos_count)
            neg_correct = (neg_out < 0).sum() / float(neg_count)
            print("Iteration={}, pos correct={}, neg correct={}".format(i,pos_correct,neg_correct))

    return weights

wts = train(pos_examples, neg_examples)
print(wts.transpose())

def plot_boundary(positive_examples, negative_examples, weights):
    if np.isclose(weights[1], 0):
        if np.isclose(weights[0], 0):
            x = y = np.array([-6, 6], dtype = 'float32')
        else:
            y = np.array([-6, 6], dtype= 'float32')
            x = -(weights[1] * y + weights[2])/weights[0]
    else:
        x = np.array([-6, 6], dtype='float32')
        y = -(weights[0] * x + weights[2])/weights[1]
    plt.xlim(-6,6)
    plt.ylim(-6,6)
    plt.plot(positive_examples[:, 0], positive_examples[:,1], 'bo')
    plt.plot(negative_examples[:, 0], negative_examples[:, 1], 'ro')
    plt.plot(x, y, 'g', linewidth=2.0)
    plt.show()

plot_boundary(pos_examples,neg_examples,wts)

def accuracy(weights, test_x, test_labels):
    res = np.dot(np.c_[test_x,np.ones(len(test_x))],weights)
    return (res.reshape(test_labels.shape)*test_labels>=0).sum()/float(len(test_labels))

def train_graph(positive_examples, negative_examples, num_iterations = 100):
    num_dims = positive_examples.shape[1]
    weights = np.zeros((num_dims, 1))
    
    pos_count = positive_examples.shape[0]
    neg_count = negative_examples.shape[0]

    report_frequency = 15
    snapshots = []
    
    for i in range(num_iterations):
        pos = random.choice(positive_examples)
        neg = random.choice(negative_examples)
        
        z = np.dot(pos, weights)
        if z < 0:
            weights = weights + pos.reshape(weights.shape)
        z = np.dot(neg, weights)
        if z >= 0:
            weights = weights - neg.reshape(weights.shape)
        if i % report_frequency == 0:
            pos_out = np.dot(positive_examples, weights)
            neg_out = np.dot(negative_examples, weights)
            pos_correct = (pos_out >= 0).sum() /float(pos_count)
            neg_correct = (neg_out < 0).sum() / float(neg_count)
            snapshots.append((np.concatenate(weights), [(pos_correct+neg_correct)/2.0, 0, 0]))
    return np.array(snapshots)

snapshots = train_graph(pos_examples, neg_examples)
def plotit(pos_examples,neg_examples,snapshots,step):
    fig = plt.figure(figsize=(10,4))
    fig.add_subplot(1, 2, 1)
    plot_boundary(pos_examples, neg_examples, snapshots[step][0])
    fig.add_subplot(1, 2, 2)
    plt.plot(np.arange(len(snapshots[:,1])), snapshots[:,1])
    plt.ylabel('Accuracy')
    plt.xlabel('Iteration')
    plt.plot(step, snapshots[step,1][0], "bo")
    plt.show()
def pl1(step): plotit(pos_examples, neg_examples, snapshots,step)
interact(pl1, step=widgets.IntSlider(value=0, min=0, max=len(snapshots)-1))
    
pos_examples_xor = np.array([1, 0, 1], [0,1,1])
neg_examples_xor = np.array([1,1,1], [0,0,1])

snapshots_xor = train_graph(pos_examples_xor, neg_examples_xor, 1000)
def pl2(step): plotit(pos_examples_xor, neg_examples_xor, snapshots_xor, step)
interact(pl2, step=widgets.IntSlider(value=0, min=0, max=len(snapshots)-1))

with gzip.open('../../data/mnist.pkl.gz', 'rb') as mnist_pickle:
    MNIST = pickle.load(mnist_pickle)

print(MNIST['Train']['Features'][0][130:180])
print(MNIST['Train']['Labels'][0])
features = MNIST['Train']['Features'].astype(np.float32) /256.0
labels = MNIST['Train']['Labels']
fig = plt.figure(figsize=(10,5))
for i in range(10):
    ax = fig.addsubplot(1,10, i + 1)
    plt.imshow(features[i].reshape(28,28))
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

: 